## 2.2.3 神經網路
[神經網路](https://pytorch.apachecn.org/docs/1.4/blitz/neural_networks_tutorial.html)

可以使用torch.nn包来构建神经网络.

我们已经介绍了autograd包，nn包则依赖于autograd包来定义模型并对它们求导。一个nn.Module包含各个层和一个forward(input)方法，该方法返回output。

例如，下面这个神经网络可以对数字进行分类：

![title](https://pytorch.org/tutorials/_images/mnist.png)

这是一个简单的前馈神经网络(feed-forward network）。它接受一个输入，然后将它送入下一层，一层接一层的传递，最后给出输出。

一个神经网络的典型训练过程如下：
- 定义包含一些可学习参数(或者叫权重）的神经网络
- 在输入数据集上迭代
- 通过网络处理输入
- 计算loss(输出和正确答案的距离）
- 将梯度反向传播给网络的参数
- 更新网络的权重，一般使用一个简单的规则：weight = weight - learning_rate * gradient


### 定義網路

- 定義網路時，需要繼承nn.Module，並實現forwadd方法
- 把網路中具有可學習參數的層放在建置(初始化)函數__init__當中
- 如果某一層不具有可學習的參數，則建議不要放在初始化函數中(例如ReLU)，
  在forward過程中使用nn.functional代替



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        # nn.Module 子類別的函數必須在構造函數中執行父類別的構造函數
        # 下式等值於nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # 卷積層1 - 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 卷積層2 - 输入图像channel：6；输出channel：16；5x5卷积核
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 仿射層/全連接層　an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 捲積　=> 啟動 => 池化
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # 透過view改變形狀，類似numpy的reshape
        x = x.view(-1, self.num_flat_features(x)) #　-1：表示根據另一維度的大小來推論。num_flat_features =>攤平
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批处理维度的其他所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### 只需要定义 forward 函数，backward函数会在使用autograd时自动定义，backward函数用来计算导数。我们可以在 forward 函数中使用任何针对张量的操作和计算。

#### 一个模型的可学习参数可以通过net.parameters()返回

In [32]:
params = list(net.parameters())
print(len(params))
print(params[0].size(),'\n')  # conv1's .weight= 6*1*5*5。torch.Size([6, 1, 5, 5])
print(params[1].size())       # conv1's bias = 6
print(params[1])

print('='*50)
print(params[2].size(),'\n')  # conv2's .weight=16*6*5*5。torch.Size([16, 6, 5, 5])
print(params[3].size())       # conv2's  bias =16
print(params[3])   

print('='*50)
print(params[8].size(),'\n')  # fc3's .weight= 10 * 84。torch.Size([10, 84]) 
print(params[9].size())       # fc3's bias = 10
print(params[9]) 


10
torch.Size([6, 1, 5, 5]) 

torch.Size([6])
Parameter containing:
tensor([-0.1417,  0.1792, -0.0025,  0.0237, -0.0462,  0.1476],
       requires_grad=True)
torch.Size([16, 6, 5, 5]) 

torch.Size([16])
Parameter containing:
tensor([ 0.0734,  0.0237, -0.0051, -0.0095, -0.0463,  0.0350,  0.0498, -0.0444,
         0.0749,  0.0226,  0.0095, -0.0681, -0.0683,  0.0029,  0.0736,  0.0079],
       requires_grad=True)
torch.Size([10, 84]) 

torch.Size([10])
Parameter containing:
tensor([ 0.1019,  0.0500,  0.0756, -0.0058,  0.0710,  0.0103,  0.0990, -0.0777,
         0.0531,  0.0349], requires_grad=True)


#### 尝试一个随机的32x32的输入。注意:这个网络(LeNet）的期待输入是32x32的张量。如果使用MNIST数据集来训练这个网络，要把图片大小重新调整到32x32。

In [2]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0072, -0.0309,  0.0789,  0.0774,  0.0062, -0.0346, -0.0668,  0.0920,
          0.1068,  0.0664]], grad_fn=<AddmmBackward>)


#### 清零所有参数的梯度缓存，然后进行随机梯度的反向传播：

In [3]:
net.zero_grad()
out.backward(torch.randn(1, 10))

>   注意：
    torch.nn只支持小批量处理(mini-batches）。整个torch.nn包只支持小批量样本的输入，不支持单个样本的输入。<br>
    比如，nn.Conv2d 接受一个4维的张量，即nSamples x nChannels x Height x Width<br>
    如果是一个单独的样本，只需要使用input.unsqueeze(0)来添加一个“假的”批大小维度。<br>
    

## 損失函數

一个损失函数接受一对(output, target)作为输入，计算一个值来估计网络的输出和目标值相差多少。

nn中有很多不同的<font color=blue size=3 >损失函数</font> 。

* nn.MSELoss是比较简单的一种，它计算输出和目标的均方误差(mean-squared error）
- nn.CrossEntropyLoss 用來計算交叉熵損失

例如：

In [9]:
output = net(input)  # 形狀為　torch.Size([1, 10])
target = torch.randn(10)  # 本例子中使用模拟数据。形狀為torch.Size([10])
target = target.view(1, -1)  # 使目标值与数据值尺寸一致，轉成二維度資料
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print(loss.item()) # 當只有一個元素時，使用loss.item()取出數值 

tensor(1.6494, grad_fn=<MseLossBackward>)
1.6493933200836182


现在，如果使用loss的.grad_fn属性跟踪反向传播过程，会看到计算图如下：

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [ ]:
當呼叫loss.back()時，該圖會動態產生並自動微分，也會自動計算圖中參數的導數

In [5]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### 反向傳播

我们只需要调用loss.backward()来反向传播误差。我们需要清零现有的梯度，否则梯度将会与已有的梯度累加。

现在，我们将调用loss.backward()，并查看conv1层的偏置(bias）在反向传播前后的梯度。

In [6]:
# 運行.backward，觀察呼叫之前與呼叫之後的grad

net.zero_grad()     # 把net中所有可學習參數的梯度清空為0

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0041,  0.0150,  0.0377,  0.0164, -0.0257,  0.0265])


### 最佳化與更新權重

最简单的更新规则是随机梯度下降法(SGD）:

weight = weight - learning_rate * gradient

我们可以使用简单的python代码来实现:

#### 用簡單的PYTHON語法實作

In [22]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # inplace 剪法

在使用神经网络时，可能希望使用各种不同的更新规则，如SGD、Nesterov-SGD、Adam、RMSProp等。为此，我们构建了一个较小的包torch.optim，它实现了所有的这些方法。使用它很简单：

In [25]:
import torch.optim as optim

# 新建優化器，指定要調整的參數與學習率
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
optimizer.zero_grad()   # 清零梯度缓存 。與net.zero_grad()效果一樣

# 計算損失函數
output = net(input)
loss = criterion(output, target)

# 反向傳播
loss.backward()

# 更新參數
optimizer.step()  

>注意：
观察梯度缓存区是如何使用optimizer.zero_grad()手动清零的。这是因为梯度是累加的，正如前面反向传播章节叙述的那样。
